In [181]:
# Import the libraries
import pandas as pd
import numpy as np

import xgboost as xgb
from sklearn.model_selection import GridSearchCV

import seaborn as sns

import category_encoders as ce
from sklearn.preprocessing import LabelEncoder

from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error

In [182]:
# Define list to store r2 and RMSE metrics
r2 = []
rmse = []

In [183]:
# Define one-hot-encoder for "prod_category" and "segment" columns
encoder = ce.OneHotEncoder(cols=["prod_category", "segment"])

In [184]:
# Read the train dataset
df = pd.read_csv("../Datasets/train.csv")
df
# df = encoder.fit_transform(df)

,id,date,year_week,product_number,reporterhq_id,prod_category,specs,display_size,segment,sales_units,inventory_units
0,202119-6909,2021-05-15,202119,6909,93,Arale,21274,13.3,Premium,2.0,35.0
1,202120-6909,2021-05-22,202120,6909,93,Arale,21274,13.3,Premium,0.0,70.0
2,202121-6909,2021-05-29,202121,6909,93,Arale,21274,13.3,Premium,3.0,137.0
3,202122-6909,2021-06-05,202122,6909,93,Arale,21274,13.3,Premium,0.0,274.0
4,202123-6909,2021-06-12,202123,6909,93,Arale,21274,13.3,Premium,0.0,333.0
...,...,...,...,...,...,...,...,...,...,...,...
15449,202317-247737,2023-04-29,202317,247737,15,Arale,10637,13.3,Premium,0.0,62.0
15450,202317-247737,2023-04-29,202317,247737,24,Arale,10637,13.3,Premium,0.0,3.0
15451,202318-247737,2023-05-06,202318,247737,3,Arale,10637,13.3,Premium,22.0,0.0
15452,202318-247737,2023-05-06,202318,247737,15,Arale,10637,13.3,Premium,68.0,78.0


In [41]:
# Group by "reporterhq_id" and "product_number"
df.groupby(["reporterhq_id", "product_number"]).sum()

C:\Users\sergi\AppData\Local\Temp\ipykernel_8568\1371560578.py:1: FutureWarning: The default value of numeric_only in DataFrameGroupBy.sum is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  df.groupby(["reporterhq_id", "product_number"]).sum()


year_week     specs  display_size  sales_units  \
reporterhq_id product_number                                                   
3             24675            23253199    556025        1529.5       5453.0   
              30597            14359501  13868714        1107.6        872.0   
              32571            21434183  23472958        1653.6        666.0   
              33558            18806580   4496550        1302.0       4130.0   
              41454            19008721   4363104        1316.0       2919.0   
...                                 ...       ...           ...          ...   
93            240828           18198563    261090        1197.0       2082.0   
              242802           18198563    348120        1197.0       1853.0   
96            129297             809160    761996          62.4        146.0   
              201348            2023107    667230         173.0        264.0   
              205296            2022827    986340         161.0        468.0   

                              inventory_units  
reporterhq_id product_number                   
3             24675                    6383.0  
              30597                    3120.0  
              32571                    4333.0  
              33558                    5400.0  
              41454                    6726.0  
...                                       ...  
93            240828                  18053.0  
              242802                  12122.0  
96            129297                    779.0  
              201348                    980.0  
              205296                   1131.0  

[299 rows x 5 columns]

In [6]:
# Auxiliary variables for data analysis
a = list(df[df.sales_units > df.inventory_units].index)
b = list(df[df.sales_units.isna()].index)
c = list(df[df.sales_units == 0].index)

In [133]:
# Define function to get previous weeks given specific index
def get_prev_weeks(df, idx):
    print(idx)
    prev_next = df.iloc[idx-10:idx+5,:]
    # display(prev_next)
    display(prev_next[prev_next['reporterhq_id'] == df['reporterhq_id'].iloc[idx]])
    # display(prev_next)
i = 0
get_prev_weeks(df,b[i])
i += 1

884


,id,date,year_week,product_number,reporterhq_id,prod_category_1,prod_category_2,prod_category_3,prod_category_4,prod_category_5,prod_category_6,prod_category_7,specs,display_size,segment_1,segment_2,segment_3,sales_units,inventory_units
874,202235-11844,2022-09-03,202235,11844,93,0,0,1,0,0,0,0,109271,15.6,0,0,1,3.0,NaN
876,202236-11844,2022-09-10,202236,11844,93,0,0,1,0,0,0,0,109271,15.6,0,0,1,4.0,NaN
878,202237-11844,2022-09-17,202237,11844,93,0,0,1,0,0,0,0,109271,15.6,0,0,1,10.0,118.0
881,202238-11844,2022-09-24,202238,11844,93,0,0,1,0,0,0,0,109271,15.6,0,0,1,32.0,70.0
884,202239-11844,2022-10-01,202239,11844,93,0,0,1,0,0,0,0,109271,15.6,0,0,1,NaN,63.0
887,202240-11844,2022-10-08,202240,11844,93,0,0,1,0,0,0,0,109271,15.6,0,0,1,0.0,96.0


In [134]:
# Data analysis of "reporterhq_id" and "product_number" fields
df[df['reporterhq_id'] == 93][df['product_number'] == 11844	].iloc[:60]

C:\Users\berna\AppData\Local\Temp\ipykernel_15672\412213131.py:1: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  df[df['reporterhq_id'] == 93][df['product_number'] == 11844	].iloc[:60]


,id,date,year_week,product_number,reporterhq_id,prod_category_1,prod_category_2,prod_category_3,prod_category_4,prod_category_5,prod_category_6,prod_category_7,specs,display_size,segment_1,segment_2,segment_3,sales_units,inventory_units
823,202144-11844,2021-11-06,202144,11844,93,0,0,1,0,0,0,0,109271,15.6,0,0,1,98.0,335.0
826,202146-11844,2021-11-20,202146,11844,93,0,0,1,0,0,0,0,109271,15.6,0,0,1,100.0,275.0
827,202147-11844,2021-11-27,202147,11844,93,0,0,1,0,0,0,0,109271,15.6,0,0,1,122.0,206.0
828,202148-11844,2021-12-04,202148,11844,93,0,0,1,0,0,0,0,109271,15.6,0,0,1,103.0,NaN
829,202149-11844,2021-12-11,202149,11844,93,0,0,1,0,0,0,0,109271,15.6,0,0,1,82.0,94.0
830,202150-11844,2021-12-18,202150,11844,93,0,0,1,0,0,0,0,109271,15.6,0,0,1,22.0,NaN
831,202151-11844,2021-12-25,202151,11844,93,0,0,1,0,0,0,0,109271,15.6,0,0,1,90.0,26.0
832,202152-11844,2022-01-01,202152,11844,93,0,0,1,0,0,0,0,109271,15.6,0,0,1,10.0,10.0
833,202201-11844,2022-01-08,202201,11844,93,0,0,1,0,0,0,0,109271,15.6,0,0,1,6.0,NaN
834,202202-11844,2022-01-15,202202,11844,93,0,0,1,0,0,0,0,109271,15.6,0,0,1,1.0,1.0


In [185]:
# One hot encode the columns of our dataset
print(df.columns)
df = encoder.fit_transform(df)

Index(['id', 'date', 'year_week', 'product_number', 'reporterhq_id',
       'prod_category', 'specs', 'display_size', 'segment', 'sales_units',
       'inventory_units'],
      dtype='object')


In [186]:
# Drop the ID column
df = df.drop(['id'],axis=1)

In [187]:
# The the week number from the year_week attribute
df['nweek'] = df['year_week'].map(lambda x: str(x)[-2:])
df

,date,year_week,product_number,reporterhq_id,prod_category_1,prod_category_2,prod_category_3,prod_category_4,prod_category_5,prod_category_6,prod_category_7,specs,display_size,segment_1,segment_2,segment_3,sales_units,inventory_units,nweek
0,2021-05-15,202119,6909,93,1,0,0,0,0,0,0,21274,13.3,1,0,0,2.0,35.0,19
1,2021-05-22,202120,6909,93,1,0,0,0,0,0,0,21274,13.3,1,0,0,0.0,70.0,20
2,2021-05-29,202121,6909,93,1,0,0,0,0,0,0,21274,13.3,1,0,0,3.0,137.0,21
3,2021-06-05,202122,6909,93,1,0,0,0,0,0,0,21274,13.3,1,0,0,0.0,274.0,22
4,2021-06-12,202123,6909,93,1,0,0,0,0,0,0,21274,13.3,1,0,0,0.0,333.0,23
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
15449,2023-04-29,202317,247737,15,1,0,0,0,0,0,0,10637,13.3,1,0,0,0.0,62.0,17
15450,2023-04-29,202317,247737,24,1,0,0,0,0,0,0,10637,13.3,1,0,0,0.0,3.0,17
15451,2023-05-06,202318,247737,3,1,0,0,0,0,0,0,10637,13.3,1,0,0,22.0,0.0,18
15452,2023-05-06,202318,247737,15,1,0,0,0,0,0,0,10637,13.3,1,0,0,68.0,78.0,18


In [188]:
# Define the "nweek" attribute as an int
df['nweek'] = df['nweek'].astype(np.int64)
df.dtypes

date                object
year_week            int64
product_number       int64
reporterhq_id        int64
prod_category_1      int64
prod_category_2      int64
prod_category_3      int64
prod_category_4      int64
prod_category_5      int64
prod_category_6      int64
prod_category_7      int64
specs                int64
display_size       float64
segment_1            int64
segment_2            int64
segment_3            int64
sales_units        float64
inventory_units    float64
nweek                int64
dtype: object

In [189]:
# Redefine the 'date' attribute as datetime
df['date'] = pd.to_datetime(df['date'])
print(df['date'][0])

2021-05-15 00:00:00


In [190]:
df

,date,year_week,product_number,reporterhq_id,prod_category_1,prod_category_2,prod_category_3,prod_category_4,prod_category_5,prod_category_6,prod_category_7,specs,display_size,segment_1,segment_2,segment_3,sales_units,inventory_units,nweek
0,2021-05-15,202119,6909,93,1,0,0,0,0,0,0,21274,13.3,1,0,0,2.0,35.0,19
1,2021-05-22,202120,6909,93,1,0,0,0,0,0,0,21274,13.3,1,0,0,0.0,70.0,20
2,2021-05-29,202121,6909,93,1,0,0,0,0,0,0,21274,13.3,1,0,0,3.0,137.0,21
3,2021-06-05,202122,6909,93,1,0,0,0,0,0,0,21274,13.3,1,0,0,0.0,274.0,22
4,2021-06-12,202123,6909,93,1,0,0,0,0,0,0,21274,13.3,1,0,0,0.0,333.0,23
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
15449,2023-04-29,202317,247737,15,1,0,0,0,0,0,0,10637,13.3,1,0,0,0.0,62.0,17
15450,2023-04-29,202317,247737,24,1,0,0,0,0,0,0,10637,13.3,1,0,0,0.0,3.0,17
15451,2023-05-06,202318,247737,3,1,0,0,0,0,0,0,10637,13.3,1,0,0,22.0,0.0,18
15452,2023-05-06,202318,247737,15,1,0,0,0,0,0,0,10637,13.3,1,0,0,68.0,78.0,18


In [191]:
# Create three new attributes to store the year, month and day information
df['year'], df['month'], df['day'] = df['date'].dt.year, df['date'].dt.month, df['date'].dt.day
df = df.drop(['date'],axis=1)
df

,year_week,product_number,reporterhq_id,prod_category_1,prod_category_2,prod_category_3,prod_category_4,prod_category_5,prod_category_6,prod_category_7,...,display_size,segment_1,segment_2,segment_3,sales_units,inventory_units,nweek,year,month,day
0,202119,6909,93,1,0,0,0,0,0,0,...,13.3,1,0,0,2.0,35.0,19,2021,5,15
1,202120,6909,93,1,0,0,0,0,0,0,...,13.3,1,0,0,0.0,70.0,20,2021,5,22
2,202121,6909,93,1,0,0,0,0,0,0,...,13.3,1,0,0,3.0,137.0,21,2021,5,29
3,202122,6909,93,1,0,0,0,0,0,0,...,13.3,1,0,0,0.0,274.0,22,2021,6,5
4,202123,6909,93,1,0,0,0,0,0,0,...,13.3,1,0,0,0.0,333.0,23,2021,6,12
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
15449,202317,247737,15,1,0,0,0,0,0,0,...,13.3,1,0,0,0.0,62.0,17,2023,4,29
15450,202317,247737,24,1,0,0,0,0,0,0,...,13.3,1,0,0,0.0,3.0,17,2023,4,29
15451,202318,247737,3,1,0,0,0,0,0,0,...,13.3,1,0,0,22.0,0.0,18,2023,5,6
15452,202318,247737,15,1,0,0,0,0,0,0,...,13.3,1,0,0,68.0,78.0,18,2023,5,6


In [192]:
# Fill the NaNs with mean values
means = df.mean(axis=0)
df_noNan = df.fillna(value=means)

In [193]:
means

year_week          202240.709202
product_number     127194.308076
reporterhq_id          42.140740
prod_category_1         0.154264
prod_category_2         0.210431
prod_category_3         0.143587
prod_category_4         0.096221
prod_category_5         0.274039
prod_category_6         0.085932
prod_category_7         0.035525
specs              112262.542837
display_size           15.163543
segment_1               0.250485
segment_2               0.495082
segment_3               0.254433
sales_units            25.974938
inventory_units        71.726222
nweek                  25.004530
year                 2022.162482
month                   6.224473
day                    15.640999
dtype: float64

In [194]:
# Separate the X and Y (target) variables
X = df_noNan.drop(columns=['inventory_units'])
Y = df_noNan['inventory_units']
X
Y

0         35.0
1         70.0
2        137.0
3        274.0
4        333.0
         ...  
15449     62.0
15450      3.0
15451      0.0
15452     78.0
15453      1.0
Name: inventory_units, Length: 15454, dtype: float64

In [195]:
# Split the dataset in train and test
X_train, X_test, y_train, y_test   = train_test_split(X,Y,shuffle=True,random_state=69)

In [196]:
# Check for NaNs in X
X_train.isna().sum()

year_week          0
product_number     0
reporterhq_id      0
prod_category_1    0
prod_category_2    0
prod_category_3    0
prod_category_4    0
prod_category_5    0
prod_category_6    0
prod_category_7    0
specs              0
display_size       0
segment_1          0
segment_2          0
segment_3          0
sales_units        0
nweek              0
year               0
month              0
day                0
dtype: int64

In [197]:
# Check for NaNs in Y
y_train.isna().sum()

0

In [198]:
# Defining our models (XGBRegressor)

# First model (GPU implementation)
xgb_gpu = xgb.XGBRegressor(n_estimators =2000,
                        learning_rate= 0.15,
                        max_depth= 20,
                        min_child_weight = 1,
                        gamma = 0.0,
                        colsample_bytree=0.4,
                        sampling_method='gradient_based',
                        objective='reg:squarederror',
                        tree_method='gpu_hist',
                        enable_categorical='True')

# First model (CPU implementation)
xgb_normal = xgb.XGBRegressor(n_estimators = 300,
                              learning_boost = 0.15,
                              max_depth = 20,
                              min_child_weight = 1,
                              gamma = 0.0,
                              colsample_bytree = 0.4)

# Fit the model
grid_result_f = xgb_normal.fit(X_train, y_train)

[03:06:35] WARNING: C:\Users\dev-admin\croot2\xgboost-split_1675461376218\work\src\learner.cc:767: 
Parameters: { "learning_boost" } are not used.



In [214]:
# More XGBRegressor models with different parameters
xgb_normal2 = xgb.XGBRegressor(n_estimators = 20,
                              learning_boost = 0.15,
                              max_depth = 20,
                              min_child_weight = 1,
                              gamma = 0.0,
                              colsample_bytree = 0.4,
                              sampling_method='gradient_based',
                              objective='reg:squarederror'# ,
                            #   gpu_id = 0,
                            #   tree_method = 'gpu_hist'
                              )

grid_result_f = xgb_normal2.fit(X_train, y_train)

[03:15:38] WARNING: C:\Users\dev-admin\croot2\xgboost-split_1675461376218\work\src\learner.cc:767: 
Parameters: { "learning_boost" } are not used.



In [206]:
# Calculate the train error (RMSE)
y_predicted = xgb_normal.predict(X_train)
rms = mean_squared_error(y_train, y_predicted, squared=False)
rms

0.002200411108286222

In [215]:
# Calculate the test error (RMSE)
y_predicted = xgb_normal2.predict(X_test)
rms = mean_squared_error(y_test, y_predicted, squared=False)
rms

47.69585528345046